**Importation of libraries**

In [2]:
import yfinance as yf
import pandas as pd
import numpy as np
import numpy as np
import requests
import io

import QuantLib as ql
import xml.etree.ElementTree as ET
from datetime import datetime, timedelta

In [3]:
#Constantes
date_format = '%d/%m/%Y'

**Importation of Yield Curve Euro**

In [17]:
DATAFLOW = "YC"
FREQ = "B"  # Daily - businessweek
REF_AREA = "U2"  # Euro area (changing composition)
CURRENCY = "EUR"
PROVIDER_FM = "4F"  # ECB
INSTRUMENT_FM = "G_N_A"  # Government bond, nominal, all issuers whose rating is triple A
PROVIDER_FM_ID = "SV_C_YM"  # Svensson model - continuous compounding - yield error minimisation
DATA_TYPE_FM = "SR_1M+SR_3M+SR_1Y+SR_5Y+SR_10Y"

#N_OBS = 600
#PARAMETERS = f"lastNObservations={N_OBS}&detail=dat"
url = f"https://data-api.ecb.europa.eu/service/data/{DATAFLOW}/{FREQ}.{REF_AREA}.{CURRENCY}.{PROVIDER_FM}.{INSTRUMENT_FM}.{PROVIDER_FM_ID}.{DATA_TYPE_FM}"

response = requests.get(url, headers={'Accept': 'text/csv'})
response.raise_for_status()
df = pd.read_csv(io.StringIO(response.text), parse_dates=['TIME_PERIOD'], index_col=['TIME_PERIOD'])


df = df[["KEY", "OBS_VALUE"]].pivot(columns="KEY", values="OBS_VALUE").rename(columns = {"YC.B.U2.EUR.4F.G_N_A.SV_C_YM.SR_10Y" : "10 Yr",
                                                "YC.B.U2.EUR.4F.G_N_A.SV_C_YM.SR_5Y" : "5 Yr",
                                                "YC.B.U2.EUR.4F.G_N_A.SV_C_YM.SR_1Y" : "1 Yr",
                                                "YC.B.U2.EUR.4F.G_N_A.SV_C_YM.SR_3M" : "3 Mo"}).reindex(columns=["3 Mo", "1 Yr", "5 Yr", "10 Yr"])

df.index = df.index.strftime(date_format)
df.index.names = ['Date']


KEY,3 Mo,1 Yr,5 Yr,10 Yr
Date,,,,
06/09/2004,2.034172,2.298838,3.457222,4.209220
07/09/2004,2.040893,2.328891,3.479518,4.209626
08/09/2004,2.044384,2.346666,3.507894,4.228419
09/09/2004,2.037111,2.308988,3.430628,4.161872
10/09/2004,2.034645,2.271566,3.374729,4.120981
13/09/2004,2.041363,2.307186,3.410319,4.138965
14/09/2004,2.037903,2.295860,3.395882,4.130650
15/09/2004,2.040883,2.297686,3.395869,4.128747
16/09/2004,2.047758,2.308354,3.404290,4.130073


**Importation of yield curve Dollar + cleaning**

In [38]:
dftaux_Dollar = pd.read_csv(filepath_or_buffer = './Courbe de taux 2023 $.csv',sep = ',', parse_dates=['Date'], index_col=['Date']).sort_index(ascending=True)
dftaux_Dollar.index = dftaux_Dollar.index.strftime(date_format)
dftaux_Dollar.head(10)

,1 Mo,2 Mo,3 Mo,4 Mo,6 Mo,1 Yr,2 Yr,3 Yr,5 Yr,7 Yr,10 Yr,20 Yr,30 Yr
Date,,,,,,,,,,,,,
03/01/2023,4.17,4.42,4.53,4.70,4.77,4.72,4.40,4.18,3.94,3.89,3.79,4.06,3.88
04/01/2023,4.20,4.42,4.55,4.69,4.77,4.71,4.36,4.11,3.85,3.79,3.69,3.97,3.81
05/01/2023,4.30,4.55,4.66,4.75,4.81,4.78,4.45,4.18,3.90,3.82,3.71,3.96,3.78
06/01/2023,4.32,4.55,4.67,4.74,4.79,4.71,4.24,3.96,3.69,3.63,3.55,3.84,3.67
09/01/2023,4.37,4.58,4.70,4.74,4.83,4.69,4.19,3.93,3.66,3.60,3.53,3.83,3.66
10/01/2023,4.41,4.62,4.73,4.77,4.85,4.74,4.24,3.94,3.72,3.67,3.61,3.91,3.74
11/01/2023,4.42,4.62,4.72,4.82,4.84,4.73,4.20,3.90,3.66,3.61,3.54,3.84,3.67
12/01/2023,4.57,4.59,4.66,4.74,4.76,4.66,4.12,3.79,3.53,3.48,3.43,3.73,3.56
13/01/2023,4.58,4.59,4.67,4.73,4.77,4.69,4.22,3.88,3.60,3.55,3.49,3.79,3.61


In [39]:
dftaux_Dollar = dftaux_Dollar[['1 Mo', '3 Mo', '1 Yr', '5 Yr', '10 Yr']]
dftaux_Dollar.head(10)

,1 Mo,3 Mo,1 Yr,5 Yr,10 Yr
Date,,,,,
03/01/2023,4.17,4.53,4.72,3.94,3.79
04/01/2023,4.20,4.55,4.71,3.85,3.69
05/01/2023,4.30,4.66,4.78,3.90,3.71
06/01/2023,4.32,4.67,4.71,3.69,3.55
09/01/2023,4.37,4.70,4.69,3.66,3.53
10/01/2023,4.41,4.73,4.74,3.72,3.61
11/01/2023,4.42,4.72,4.73,3.66,3.54
12/01/2023,4.57,4.66,4.66,3.53,3.43
13/01/2023,4.58,4.67,4.69,3.60,3.49


In [40]:
for x in ['1 Mo', '3 Mo', '1 Yr', '5 Yr', '10 Yr']:
    dftaux_Dollar[f'Return_YC_Euro_{x}'] = dftaux_Dollar[x] - dftaux_Dollar[x].shift(1)
dftaux_Dollar.dropna(inplace=True)

dftaux_Dollar.head(10)

,1 Mo,3 Mo,1 Yr,5 Yr,10 Yr,Return_YC_Euro_1 Mo,Return_YC_Euro_3 Mo,Return_YC_Euro_1 Yr,Return_YC_Euro_5 Yr,Return_YC_Euro_10 Yr
Date,,,,,,,,,,
04/01/2023,4.20,4.55,4.71,3.85,3.69,0.03,0.02,-0.01,-0.09,-0.10
05/01/2023,4.30,4.66,4.78,3.90,3.71,0.10,0.11,0.07,0.05,0.02
06/01/2023,4.32,4.67,4.71,3.69,3.55,0.02,0.01,-0.07,-0.21,-0.16
09/01/2023,4.37,4.70,4.69,3.66,3.53,0.05,0.03,-0.02,-0.03,-0.02
10/01/2023,4.41,4.73,4.74,3.72,3.61,0.04,0.03,0.05,0.06,0.08
11/01/2023,4.42,4.72,4.73,3.66,3.54,0.01,-0.01,-0.01,-0.06,-0.07
12/01/2023,4.57,4.66,4.66,3.53,3.43,0.15,-0.06,-0.07,-0.13,-0.11
13/01/2023,4.58,4.67,4.69,3.60,3.49,0.01,0.01,0.03,0.07,0.06
17/01/2023,4.60,4.71,4.67,3.60,3.53,0.02,0.04,-0.02,0.00,0.04


In [105]:
dftaux_Dollar.head(10)

,1 Mo,3 Mo,1 Yr,5 Yr,10 Yr,Return_YC_Euro_1 Mo,Return_YC_Euro_3 Mo,Return_YC_Euro_1 Yr,Return_YC_Euro_5 Yr,Return_YC_Euro_10 Yr
Date,,,,,,,,,,
04/01/2023,4.20,4.55,4.71,3.85,3.69,0.03,0.02,-0.01,-0.09,-0.10
05/01/2023,4.30,4.66,4.78,3.90,3.71,0.10,0.11,0.07,0.05,0.02
06/01/2023,4.32,4.67,4.71,3.69,3.55,0.02,0.01,-0.07,-0.21,-0.16
09/01/2023,4.37,4.70,4.69,3.66,3.53,0.05,0.03,-0.02,-0.03,-0.02
10/01/2023,4.41,4.73,4.74,3.72,3.61,0.04,0.03,0.05,0.06,0.08
11/01/2023,4.42,4.72,4.73,3.66,3.54,0.01,-0.01,-0.01,-0.06,-0.07
12/01/2023,4.57,4.66,4.66,3.53,3.43,0.15,-0.06,-0.07,-0.13,-0.11
13/01/2023,4.58,4.67,4.69,3.60,3.49,0.01,0.01,0.03,0.07,0.06
17/01/2023,4.60,4.71,4.67,3.60,3.53,0.02,0.04,-0.02,0.00,0.04


**Importation of stock indices from Yahoo Finance + Cleaning**

S&P 500


In [63]:
sp500_data = yf.download('^GSPC', start='2013-10-24', end='2023-10-24', progress=False)
sp500_data.head(10)

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2013-10-24,1747.479980,1753.939941,1745.500000,1752.069946,1752.069946,3671700000
2013-10-25,1756.010010,1759.819946,1752.449951,1759.770020,1759.770020,3175720000
2013-10-28,1759.420044,1764.989990,1757.670044,1762.109985,1762.109985,3282300000
2013-10-29,1762.930054,1772.089966,1762.930054,1771.949951,1771.949951,3358460000
2013-10-30,1772.270020,1775.219971,1757.239990,1763.310059,1763.310059,3523040000
2013-10-31,1763.239990,1768.530029,1755.719971,1756.540039,1756.540039,3826530000
2013-11-01,1758.699951,1765.670044,1752.699951,1761.640015,1761.640015,3686290000
2013-11-04,1763.400024,1768.780029,1761.560059,1767.930054,1767.930054,3194870000
2013-11-05,1765.670044,1767.030029,1755.760010,1762.969971,1762.969971,3516680000


In [64]:
print(sp500_data.columns)

Index(['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume'], dtype='object')


In [65]:
sp500_data.reset_index(inplace=True)  # Réinitialise l'index pour obtenir la colonne "Date"
sp500_data = sp500_data[["Date", "Adj Close"]].rename(columns={'Adj Close': 'sp500'})

sp500_data['Date'] = pd.to_datetime(sp500_data['Date']).dt.strftime(date_format)

In [66]:
sp500_data.head(10)

,Date,sp500
0,24/10/2013,1752.069946
1,25/10/2013,1759.770020
2,28/10/2013,1762.109985
3,29/10/2013,1771.949951
4,30/10/2013,1763.310059
5,31/10/2013,1756.540039
6,01/11/2013,1761.640015
7,04/11/2013,1767.930054
8,05/11/2013,1762.969971
9,06/11/2013,1770.489990


Eurostoxx50

In [68]:
Eurostoxx50_data = yf.download('^STOXX50E', start='2013-10-24', end='2023-10-24', progress=False)
print(Eurostoxx50_data.head())
Eurostoxx50_data.size

                   Open         High          Low        Close    Adj Close  \
Date                                                                          
2013-10-24  3024.689941  3038.959961  3022.280029  3038.959961  3038.959961   
2013-10-25  3027.840088  3043.840088  3021.010010  3034.500000  3034.500000   
2013-10-28  3041.159912  3045.139893  3009.800049  3022.040039  3022.040039   
2013-10-29  3019.649902  3050.739990  3017.729980  3050.639893  3050.639893   
2013-10-30  3049.090088  3073.879883  3035.020020  3040.689941  3040.689941   

              Volume  
Date                  
2013-10-24  53819900  
2013-10-25  49404800  
2013-10-28  51393200  
2013-10-29  59850600  
2013-10-30  60923400  


15054

In [69]:
Eurostoxx50_data.reset_index(inplace=True)  # Réinitialise l'index pour obtenir la colonne "Date"
Eurostoxx50_data = Eurostoxx50_data[["Date", "Adj Close"]].rename(columns={'Adj Close': 'Eurostoxx50'})

Eurostoxx50_data['Date'] = pd.to_datetime(Eurostoxx50_data['Date']).dt.strftime(date_format)

In [70]:
Eurostoxx50_data.head(10)

,Date,Eurostoxx50
0,24/10/2013,3038.959961
1,25/10/2013,3034.500000
2,28/10/2013,3022.040039
3,29/10/2013,3050.639893
4,30/10/2013,3040.689941
5,31/10/2013,3067.949951
6,01/11/2013,3052.139893
7,04/11/2013,3061.179932
8,05/11/2013,3035.919922
9,06/11/2013,3056.399902


**Euro/Dollar exchange rate importation + cleaning**

In [91]:
dftaux = pd.read_csv(filepath_or_buffer = './Taux de change euro dollar.csv',sep = ',')[['_TIME_PERIOD', '_OBS_VALUE']].rename(columns={'_TIME_PERIOD': 'Date', '_OBS_VALUE': 'TauxdeChange'})
dftaux['Date'] = pd.to_datetime(dftaux['Date']).dt.strftime(date_format)
dftaux.head(10)

,Date,TauxdeChange
0,04/01/1999,1.1789
1,05/01/1999,1.1790
2,06/01/1999,1.1743
3,07/01/1999,1.1632
4,08/01/1999,1.1659
5,11/01/1999,1.1569
6,12/01/1999,1.1520
7,13/01/1999,1.1744
8,14/01/1999,1.1653
9,15/01/1999,1.1626


In [92]:
dftaux = pd.merge(dftaux, sp500_data, on='Date', how='inner')
dftaux = pd.merge(dftaux, Eurostoxx50_data, on='Date', how='inner')
dftaux.set_index('Date', inplace=True)

In [94]:
dftaux.head(10)

,TauxdeChange,sp500,Eurostoxx50
Date,,,
24/10/2013,1.3805,1752.069946,3038.959961
25/10/2013,1.3777,1759.770020,3034.500000
28/10/2013,1.3784,1762.109985,3022.040039
29/10/2013,1.3768,1771.949951,3050.639893
30/10/2013,1.3755,1763.310059,3040.689941
31/10/2013,1.3641,1756.540039,3067.949951
01/11/2013,1.3505,1761.640015,3052.139893
04/11/2013,1.3506,1767.930054,3061.179932
05/11/2013,1.3494,1762.969971,3035.919922


**Yields calculation**

In [95]:
dftaux['Log_Return_SP500'] = np.log(dftaux['sp500'] / dftaux['sp500'].shift(1))

In [97]:
dftaux.head(10)

,TauxdeChange,sp500,Eurostoxx50,Log_Return_SP500
Date,,,,
24/10/2013,1.3805,1752.069946,3038.959961,NaN
25/10/2013,1.3777,1759.770020,3034.500000,0.004385
28/10/2013,1.3784,1762.109985,3022.040039,0.001329
29/10/2013,1.3768,1771.949951,3050.639893,0.005569
30/10/2013,1.3755,1763.310059,3040.689941,-0.004888
31/10/2013,1.3641,1756.540039,3067.949951,-0.003847
01/11/2013,1.3505,1761.640015,3052.139893,0.002899
04/11/2013,1.3506,1767.930054,3061.179932,0.003564
05/11/2013,1.3494,1762.969971,3035.919922,-0.002810


In [98]:
dftaux['Log_Return_Eurostoxx50'] = np.log(dftaux['Eurostoxx50'] / dftaux['Eurostoxx50'].shift(1))

In [99]:
dftaux = dftaux.dropna()

In [101]:

dftaux.head(10)

,TauxdeChange,sp500,Eurostoxx50,Log_Return_SP500,Log_Return_Eurostoxx50
Date,,,,,
25/10/2013,1.3777,1759.770020,3034.500000,0.004385,-0.001469
28/10/2013,1.3784,1762.109985,3022.040039,0.001329,-0.004115
29/10/2013,1.3768,1771.949951,3050.639893,0.005569,0.009419
30/10/2013,1.3755,1763.310059,3040.689941,-0.004888,-0.003267
31/10/2013,1.3641,1756.540039,3067.949951,-0.003847,0.008925
01/11/2013,1.3505,1761.640015,3052.139893,0.002899,-0.005167
04/11/2013,1.3506,1767.930054,3061.179932,0.003564,0.002957
05/11/2013,1.3494,1762.969971,3035.919922,-0.002810,-0.008286
06/11/2013,1.3517,1770.489990,3056.399902,0.004256,0.006723


In [102]:
dftaux['Log_Return_TauxdeChange'] = np.log(dftaux['TauxdeChange'] / dftaux['TauxdeChange'].shift(1))

In [103]:
dftaux = dftaux.dropna()

In [104]:
dftaux.head(10)

,TauxdeChange,sp500,Eurostoxx50,Log_Return_SP500,Log_Return_Eurostoxx50,Log_Return_TauxdeChange
Date,,,,,,
28/10/2013,1.3784,1762.109985,3022.040039,0.001329,-0.004115,0.000508
29/10/2013,1.3768,1771.949951,3050.639893,0.005569,0.009419,-0.001161
30/10/2013,1.3755,1763.310059,3040.689941,-0.004888,-0.003267,-0.000945
31/10/2013,1.3641,1756.540039,3067.949951,-0.003847,0.008925,-0.008322
01/11/2013,1.3505,1761.640015,3052.139893,0.002899,-0.005167,-0.010020
04/11/2013,1.3506,1767.930054,3061.179932,0.003564,0.002957,0.000074
05/11/2013,1.3494,1762.969971,3035.919922,-0.002810,-0.008286,-0.000889
06/11/2013,1.3517,1770.489990,3056.399902,0.004256,0.006723,0.001703
07/11/2013,1.3365,1747.150024,3042.979980,-0.013270,-0.004400,-0.011309


***Génération des scénarios avec la méthode de bootstrapping***

In [106]:
#Scénarios de l'eurostoxx50
nb_scenarios = 1000
scenarios_bootstrap_Eurostoxx50 = []
for i in range(nb_scenarios):
    nv_scenario = np.random.choice(dftaux['Log_Return_Eurostoxx50'] , len(dftaux) , replace=True)    
    scenarios_bootstrap_Eurostoxx50.append(nv_scenario)
print(scenarios_bootstrap_Eurostoxx50)

scenarios_totaux=0
for i in range(len(scenarios_bootstrap_Eurostoxx50)):
    scenarios_totaux=scenarios_totaux+scenarios_bootstrap_Eurostoxx50[i]
print(scenarios_totaux)

#Scénarios du taux de change
nb_scenarios = 1000
scenarios_bootstrap_TauxdeChange = []
for i in range(nb_scenarios):
    nv_scenario = np.random.choice(dftaux['Log_Return_TauxdeChange'] , len(dftaux) , replace=True)    
    scenarios_bootstrap_TauxdeChange.append(nv_scenario)
print(scenarios_bootstrap_TauxdeChange)

scenarios_totaux=0
for i in range(len(scenarios_bootstrap_TauxdeChange)):
    scenarios_totaux=scenarios_totaux+scenarios_bootstrap_TauxdeChange[i]
print(scenarios_totaux)

#Scénarios de l'indice SP500
nb_scenarios = 1000
scenarios_bootstrap_SP500 = []
for i in range(nb_scenarios):
    nv_scenario = np.random.choice(dftaux['Log_Return_TauxdeChange'] , len(dftaux) , replace=True)    
    scenarios_bootstrap_SP500.append(nv_scenario)
print(scenarios_bootstrap_SP500)

scenarios_totaux=0
for i in range(len(scenarios_bootstrap_SP500)):
    scenarios_totaux=scenarios_totaux+scenarios_bootstrap_SP500[i]
print(scenarios_totaux)


[array([-0.00643739, -0.00148815, -0.00530301, ..., -0.0016837 ,
        0.01405404, -0.00435586]), array([ 0.00168807, -0.03163027,  0.0024192 , ...,  0.0053982 ,
       -0.00051167, -0.00157129]), array([-0.0269318 , -0.00389409,  0.00094759, ..., -0.01944529,
        0.00254533,  0.00258477]), array([ 0.0029253 , -0.01204047,  0.00605521, ..., -0.00841997,
        0.003223  ,  0.00509029]), array([-0.00953183,  0.00313852,  0.00546299, ..., -0.00810692,
       -0.01841761, -0.00637079]), array([ 0.00114447, -0.00101985,  0.0014487 , ...,  0.01126628,
        0.00866607,  0.01189777]), array([-0.01084493,  0.00040765, -0.01155042, ..., -0.01204821,
       -0.00022835,  0.01283906]), array([ 0.00379952, -0.00119331,  0.00116515, ..., -0.01120101,
       -0.00440043, -0.01776492]), array([-0.02826091,  0.007477  , -0.01473983, ..., -0.03163027,
        0.00368401, -0.00065012]), array([-0.00378456,  0.01297098,  0.00189197, ...,  0.00041586,
       -0.01241352, -0.0096886 ]), array([-0

***Swap pricing***

In [ ]:
# Charger le fichier XML
xml_data = """
<instrumentlist xmlns="http://www.statpro.net/xml/structure/1.0"><http://www.statpro.net/xml/structure/1.0%22%3e>
  <!-- plain swap -->
  <instrument>
    <code>TEST0183</code>
    <codetype>test-dp</codetype>
    <description>Sample interest-rate swap</description>
    <currency>EUR</currency>
    <swap>
      <counterpartycode>Foo</counterpartycode>
      <fixedrateleg>
        <side>Pay</side>
        <startdate>2008-03-18</startdate>
        <maturitydate>2018-03-17</maturitydate>
        <settlementdays>2</settlementdays>
        <calendar>TARGET</calendar>
        <rollingconvention>Following</rollingconvention>
        <frequency>1</frequency>
        <adjusted>false</adjusted>
        <daycount>30/360</daycount>
        <notionals>100.0</notionals>
        <couponrates>0.04</couponrates>
      </fixedrateleg>
      <floatingrateleg>
        <side>Receive</side>
        <startdate>2008-03-18</startdate>
        <maturitydate>2018-03-17</maturitydate>
        <settlementdays>2</settlementdays>
        <calendar>TARGET</calendar>
        <rollingconvention>ModifiedFollowing</rollingconvention>
        <frequency>2</frequency>
        <adjusted>true</adjusted>
        <daycount>Act/360</daycount>
        <notionals>100.0</notionals>
        <referenceindex>EUR-Euribor-6-m</referenceindex>
        <fixingdays>2</fixingdays>
        <fixingtype>Advance</fixingtype>
        <spreads>0.002</spreads>
      </floatingrateleg>
    </swap>
  </instrument>
</instrumentlist>

"""



In [ ]:
# Dates
valuation_date = ql.Date(2010, 1, 1)
start_date = ql.Date(2008, 3, 18)
end_date = ql.Date(2018, 3, 17)

# Market data
discount_curve = ql.YieldTermStructureHandle(ql.FlatForward(valuation_date, ql.QuoteHandle(ql.SimpleQuote(0.03)), ql.Actual360()))

# Fixed Leg
fixed_rate = 0.04
fixed_leg = ql.FixedRateLeg(ql.Schedule(start_date, end_date, ql.Period(ql.Semiannual), ql.TARGET(), ql.ModifiedFollowing, ql.ModifiedFollowing, ql.DateGeneration.Backward, False), ql.Actual360(), [100.0], [fixed_rate])

# Floating Leg
floating_leg = ql.IborLeg([100.0], ql.Euribor(ql.Period(6, ql.Months), ql.YieldTermStructureHandle(ql.FlatForward(valuation_date, ql.QuoteHandle(ql.SimpleQuote(0.025)), ql.Actual360())))

# Swap
swap = ql.Swap(fixed_leg, floating_leg)
swap.setPricingEngine(ql.DiscountingSwapEngine(discount_curve))

# Calcul de la valeur du swap
npv = swap.NPV()
print("La valeur actuelle nette (NPV) du swap est :", npv)

In [ ]:
root = ET.fromstring(xml_data)


In [ ]:
root = ET.fromstring(xml_data)

# Accéder aux éléments du fichier XML
code = root.find(".//code").text
description = root.find(".//description").text
currency = root.find(".//currency").text

# Pour accéder aux détails de la jambe fixe et de la jambe variable, vous pouvez utiliser des boucles ou des recherches similaires.
fixed_leg = root.find(".//fixedrateleg")
floating_leg = root.find(".//floatingrateleg")

# Accéder aux éléments des jambes fixe et variable
fixed_start_date = fixed_leg.find(".//startdate").text
fixed_maturity_date = fixed_leg.find(".//maturitydate").text
# Vous pouvez faire de même pour les autres éléments.

# Traiter les données selon vos besoins

# Exemple : Imprimer certaines informations
print("Code:", code)
print("Description:", description)
print("Currency:", currency)
print("Fixed Start Date:", fixed_start_date)
print("Fixed Maturity Date:", fixed_maturity_date)

In [ ]:
def fixed_leg(date_calc, maturity, coupon, freq, YC):
    cashflows = []

    current_date = date_calc
    while current_date <= maturity:
        fixed_cashflow = YC(current_date) * coupon / freq
        cashflows.append({
            'date': current_date,
            'cashflow': fixed_cashflow
        })
        current_date += timedelta(days=int(360 / freq))

    return cashflows

def float_rate(date_calc, date, maturity, YC):
    # Calcul du taux forward entre date_calc et date
    rate_date_calc = YC(date_calc)
    rate_date = YC(date)
    
    # Vérifier si la différence entre les dates est non nulle
    if (date - date_calc).days != 0:
        # Supposons une interpolation linéaire pour simplifier
        forward_rate = rate_date_calc + (rate_date - rate_date_calc) / ((date - date_calc).days / 360)
        return forward_rate
    else:
        # Gérer le cas où la différence est nulle
        return rate_date



def float_leg(date_calc, maturity, float_spread, freq, YC):
    cashflows = []

    current_date = date_calc
    while current_date <= maturity:
        float_cashflow = YC(current_date) + float_spread / freq
        cashflows.append({
            'date': current_date,
            'cashflow': float_cashflow
        })
        current_date += timedelta(days=int(360 / freq))

    return cashflows

def calculate_cashflows(fixed_leg, floating_leg):
    cashflows = []

    for fixed_cf, float_cf in zip(fixed_leg, floating_leg):
        cashflows.append({
            'date': fixed_cf['date'],
            'fixed_leg_cashflow': fixed_cf['cashflow'],
            'floating_leg_cashflow': float_cf['cashflow']
        })

    return cashflows

def present_value(cashflows):
    pv = 0
    for cf in cashflows:
        pv += cf['fixed_leg_cashflow'] - cf['floating_leg_cashflow']
    return pv

# Paramètres du swap
date_calc = datetime.strptime("2008-03-18", "%Y-%m-%d")
maturity = datetime.strptime("2018-03-17", "%Y-%m-%d")

# Fonction de courbe des taux (yield curve)
def yield_curve(date):
    # Vous devrez remplacer cela par une logique réelle pour obtenir les taux du marché
    # Pour cet exemple, je retourne simplement 0.03 pour toutes les dates
    return 0.03

# Calcul des flux de trésorerie pour la jambe fixe
fixed_leg_cashflows = fixed_leg(date_calc, maturity, 0.04, 1, yield_curve)

# Calcul des flux de trésorerie pour la jambe variable
float_spread = 0.002
float_leg_cashflows = float_leg(date_calc, maturity, float_spread, 2, yield_curve)

# Calcul du taux variable pour chaque date de paiement
float_rates = [float_rate(date_calc, cf['date'], maturity, yield_curve) for cf in float_leg_cashflows]

# Ajout du taux variable aux flux de trésorerie de la jambe variable
for i in range(len(float_leg_cashflows)):
    float_leg_cashflows[i]['cashflow'] += float_rates[i]

# Calcul des flux de trésorerie combinés
cashflows = calculate_cashflows(fixed_leg_cashflows, float_leg_cashflows)

# Calcul de la valeur actuelle nette (présent value)
pv = present_value(cashflows)

print(f"La valeur actuelle nette du swap est : {pv:.2f} EUR")
